In [14]:
import numpy as np
import pandas as pd
import os
import random

from mcda.outranking.electre import Electre3
from mcda.outranking.promethee import Promethee2, Promethee1
from mcda.functions import UShapeFunction, VShapeFunction, LevelFunction, GaussianFunction
from mcda import *
from mcda.matrices import *
from mcda.relations import *
from mcda.scales import *
from mcda.values import CommensurableValues

In [15]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [16]:
from helper_functions import dict_to_matrix, dict_to_matrix_with_prefix, net_flow_score, net_flow_score_pos_neg, create_preference_matrix, outranking_ranking, resolve_matrix_preferences, visualize_outranking
import json

In [48]:
def generate_outrankings(input_dir, output_dir):
    """
    Generate outranking results based on ELECTRE III, PROMETHEE I, and crisp outrankings.

    Parameters:
    - input_dir: str - Directory containing input CSV files.
    - output_dir: str - Directory to save the outranking results.
    """
    os.makedirs(output_dir, exist_ok=True)

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".csv") and not file.endswith("_weights.csv"):
                file_path = os.path.join(root, file)
                weights_file = file_path.replace(".csv", "_weights.csv")

                # Load performances and weights
                data = pd.read_csv(file_path)

                weights_data = pd.read_csv(weights_file)

                performances = data.values
                weights = weights_data["Weight"].values
                alt_len = len(performances)
                crit_len = len(weights)

                optimization = weights_data["Optimization"].values.tolist()
                #lambda_value = weights_data["Lambda"][0]  # Lambda is the same for all criteria
                q_vals = weights_data["Q"].values
                p_vals = weights_data["P"].values
                v_vals = weights_data["V"].values
                cutoff = weights_data["cutoff"][0]

                # Get directory structure
                rel_dir = os.path.relpath(root, input_dir)
                os.makedirs(rel_dir, exist_ok=True)

                scales = {}
                preference_func_list = {}
                W = {}
                I = {}
                P = {}
                V = {}
                for i in range(crit_len):
                    scales[i] = QuantitativeScale(0, 1, preference_direction= ( MAX if optimization[i] == 'max' else MIN))
                    W[i] = weights[i]
                    I[i] = q_vals[i]
                    P[i] = p_vals[i]
                    V[i] = v_vals[i]
                    preference_func_list[i] = VShapeFunction(p=p_vals[i], q=q_vals[i])

                dataset = PerformanceTable(performances, 
                                           alternatives=[str(i) for i in range(alt_len)],
                                           scales=scales)

                promethee1 = Promethee1(dataset, W, preference_func_list)
                promethee2 = Promethee2(dataset, W, preference_func_list)
                electre3 = Electre3(performance_table=dataset, 
                                    criteria_weights=W, 
                                    indifference_thresholds=I, 
                                    preference_thresholds=P, 
                                    veto_thresholds=V)

                promethee_matrix = promethee1.preferences(promethee1.partial_preferences()).data
                electre_matrix = electre3.construct().data
                electre_binary_matrix = (electre_matrix >= cutoff).astype(int)

                promethee1_rank = promethee1.rank().outranking_matrix.data.values
                promethee2_rank = PreferenceStructure().from_ranking(promethee2.rank()).outranking_matrix.data.values
                electre3_rank_nfs = net_flow_score(electre_binary_matrix.values)
                electre3_rank_nfs_pos_neg = create_preference_matrix(net_flow_score_pos_neg(electre_binary_matrix.values))
               
                descending_distillate = electre3.distillation(
                    electre3.credibility(),
                )
                ascending_distillate = electre3.distillation(
                    electre3.credibility(),
                    ascending=True
                )       

                # Electre 3 median ranking
                electre3_median_rank = resolve_matrix_preferences(electre3.rank().data.values,outranking_ranking(descending_distillate.data.values), outranking_ranking(ascending_distillate.data.values))

                electre3_final_ranking = electre3.rank().data.values


                results = {}
                results['promethee_matrix'] = promethee_matrix.values.tolist()
                results['electre_matrix'] = electre_matrix.values.tolist()
                results['electre_binary_matrix'] = electre_binary_matrix.values.tolist()
                results['promethee1_rank'] = promethee1_rank.tolist()
                results['promethee2_rank'] = promethee2_rank.tolist()
                results['electre3_rank_nfs'] = electre3_rank_nfs.tolist()
                results['electre3_rank_nfs_pos_neg'] = electre3_rank_nfs_pos_neg.tolist()
                results['electre3_median_rank'] = electre3_median_rank.tolist()
                results['electre3_final_ranking'] = electre3_final_ranking.tolist()

                full_output_path = os.path.join(output_dir, os.path.relpath(root, input_dir))
                os.makedirs(full_output_path, exist_ok=True)

                results_path = os.path.join(output_dir, rel_dir, f"{file.split('.')[0]}_results.json")
                
                os.makedirs(os.path.dirname(results_path), exist_ok=True)

                with open(results_path, "w+") as outfile: 
                    json.dump(results, outfile, indent=4)

                

In [50]:
outranking_parameters = {
    "input_dir": "data",
    "output_dir": "rankings_and_matrix"
}

generate_outrankings(
    input_dir=outranking_parameters["input_dir"],
    output_dir=outranking_parameters["output_dir"]
)